# preprocess_threads

ダウンロードしたスレッドタイトルを正規化して１ファイルに纏める

|パラメータ名|内容|
|:--|:--|
| `THREAD_FILTER_TH_MIN_RESPONSE` | 学習に利用するスレッドタイトルの最小レス数 |
| `THREAD_FILTER_NG_PHRASES` | スレッドタイトルのNG文字列、この文字列が含まれるスレッドタイトルは学習に利用しない。 |
| `IGNORE_PHRASES` | スレッドタイトルから除外する文字列 |
| `SPECIAL_TOKEN_MAP` | 前処理で変換するの特殊文字 |
| `SPECIAL_TOKEN_REGEX_MAP` | 前処理で変換するの特殊文字（正規表現） |

## 入力ファイル

* `data/*.tsv`

## 出力ファイル

* `normalized_titles.txt`: 正規化済のスレッドタイトル、１行に1スレッドタイトル
* `normalizer.pickle`: 正規化を行うオブジェクト

In [1]:
import glob
import pickle
import re

In [2]:
THREAD_FILTER_TH_MIN_RESPONSE = 5
# アルファベットはlowercaseにする
THREAD_FILTER_NG_PHRASES = [
    "id:", "http"
]
IGNORE_PHRASES = [
    "[無断転載禁止]©2ch.net",
    "\\n", "\n", "・", "、", "。", ",", ".",
    "★", "☆", "○", "●", "◎" "◯ ", "■", "□"
]
SPECIAL_TOKEN_MAP = {
    "彡(゚)(゚)": "<nanjmin>",
    "彡(^)(^)": "<nikojmin>",
    "彡(;)(;)": "<nakijmin>",
    "（ヽ´ん`）": "<kenmoukun>",
    "(ヽ ́ん`)": "<kenmoukun>",
    "(ヽ’ん`)": "<kenmoukun>",
    "(ヽ'ん`)": "<kenmoukun>",
    "( ヽ ́ん`)": "<kenmoukun>",
    "(´・ω・`)": "<genjumin>",
    "(´・ω・｀)": "<genjumin>",
    "( ́;ω;`)": "<nakigenjumin>",
    "(*^◯^*)": "<pojihamekun>",
    "(*^^*)": "<pojihamekun>",
    "( ́ω`)": "<kao1>",
}
SPECIAL_TOKEN_REGEX_MAP = {
    re.compile("ww+"): "<kusa>"
}

In [3]:
# ダウンロードしたファイルからスレッドタイトルリストを作成する
titles = []
for fpath in glob.glob("data/*.tsv"):
    with open(fpath) as f:
        for line in f:
            text = line.rstrip()
            elem = re.split(r'\t+', text)
            # パースに失敗したものは除外する
            if len(elem) != 2:
                continue
            thread_title, num_response = elem
            # NGフレーズが含まれる場合除外
            for phrase in THREAD_FILTER_NG_PHRASES:
                if phrase in thread_title:
                    continue
            # レス数が少ないものを除外
            if int(num_response) < THREAD_FILTER_TH_MIN_RESPONSE:
                continue
            titles.append(thread_title)

In [4]:
print(f"スレッドタイトル数: {len(titles)}")

スレッドタイトル数: 2016289


In [ ]:
# スレッドタイトル例
print(titles[:3])

In [6]:
from scripts.normalize import ThreadTitleNormalizer
normalizer = ThreadTitleNormalizer(IGNORE_PHRASES, SPECIAL_TOKEN_MAP, SPECIAL_TOKEN_REGEX_MAP)

In [7]:
text = "彡(゚)(゚)「円周率は3ｗｗｗ」ワイ「🤔」 [無断転載禁止]©2ch.net"
normalizer.normalize(text)

'<nanjmin>「円周率は3<kusa>」ワイ「🤔」'

In [9]:
# 正規化済みのスレッドタイトルをテキストファイルに保存する
with open("normalized_titles.txt", "w") as f:
    for title in titles:
        normalized = normalizer.normalize(title)
        if len(normalized) > 0:
            f.write(f"{normalized}\n")

In [10]:
# NormalizerをPickleで保存する
with open("normalizer.pickle", "wb") as f:
    pickle.dump(normalizer, f, pickle.HIGHEST_PROTOCOL)